In [1]:
#import necessary packages
import pandas as pd
import time
import tkinter as tk
from tkinter import *
from tkinter import scrolledtext as tkst
from tkinter.ttk import *
import pyperclip as pc
import inputs
import cx_Oracle
from tqdm import tqdm
import pulp
from pulp import *
import random


#Build Root window
root = tk.Tk()
root.geometry('825x570')
root.title("Heat Treat Scheduling Tool")
Nrows = 25
Ncols = 3

cellre = re.compile(r'\b[A-Z][0-9]\b')

In [2]:
def cellname(i, j):
    return f'{chr(ord("A")+j)}{i+1}'

In [3]:
class Cell():
    '''Contains all code for cell operations: focus, select, edit, move, copy, paste'''
    def __init__(self, i, j, siblings, parent):
        self.row = i
        self.col = j
        self.siblings = siblings
        self.name = cellname(i, j)
        self.formula = self
        self.value = ''
        # Dependencies - must be updated if this cell changes
        self.deps = set()
        # Requirements - values required for this cell to calculate
        self.reqs = set()
        self.var = tk.StringVar()
        #define configuration / bind parameters for entry boxes ('Cells')
        entry = self.widget = tk.Entry(parent, textvariable=self.var,
                                       readonlybackground="white", width=10, relief=FLAT,
                                       borderwidth=1, highlightcolor="green",
                                       highlightthickness=1, justify="right")
        #entry.config({"background": "#FD7B35"})
        entry.bind('<FocusIn>', self.deselect)
        entry.bind('<FocusOut>', self.deselect)
        entry.bind('<Double-Button-1>', self.edit)
        entry.bind('<Key>', self.edit)
        entry.bind('<Return>', self.move(1, 0))
        entry.bind('<Up>', self.move(-1, 0))
        entry.bind('<Down>', self.move(+1, 0))
        entry.bind('<Left>', self.move(0, -1))
        entry.bind('<Right>', self.move(0, 1))
        entry.bind('<Control-v>', self.paste)
   
    def copy(self):
        list = []
        for i in range(19):
            inner_list = []
            for j in range(3):
                item = self.siblings[cellname(i, j)].widget.get()
                inner_list.append(item)
            list.append(inner_list)
        return list
                
    def paste(self, event):
        list_to_paste = []
        input = pc.paste()
        str_input = str(input)
        line_list = str_input.split('\r\n')
        #add pasted input to list of lists; separating by tabs and new lines
        for line in line_list[0:-1]:
            word_list = []
            words = line.split('\t')
            for word in words:
                word_list.append(word)
            list_to_paste.append(word_list)
        i = self.row
        #for loops to go through list of lists and paste into cells
        for list in list_to_paste:
            j = self.col
            for item in list:
                self.siblings[cellname(i, j)].widget.config(state='normal')
                self.siblings[cellname(i, j)].widget.delete(0, END)
                self.siblings[cellname(i, j)].widget.insert(END,item)
                self.siblings[cellname(i, j)].widget.config(state='readonly')
                j = j + 1
            i = i + 1

    def move(self, rowadvance, coladvance):
        targetrow = (self.row + rowadvance) % Nrows
        targetcol = (self.col + coladvance) % Ncols
        def focus(event):
            targetwidget = self.siblings[cellname(targetrow, targetcol)].widget
            targetwidget.focus()
        return focus

    def propagate(self):
        for d in self.deps:
            #self.siblings[d].calculate()
            self.siblings[d].propagate()

    def deselect(self, event):
        self.widget.config(state='readonly')
    
    def edit(self, event):
        event_type = str(event.type)
        config_dict = self.widget.config()
        cell_state = config_dict['state'][4]
        print(cell_state)
        if (event_type == 'ButtonPress'):
            self.widget.config(state='normal')
            self.widget.select_range(0, tk.END)
        elif (event_type == 'KeyPress' and cell_state =='readonly'):
            self.widget.config(state='normal')
            self.widget.select_range(0, tk.END)
        else:
            self.widget.config(state='normal')

    def update(self, event):
        self.formula = self.var.get()
        self.propagate()
        # If this was after pressing Return, keep showing the formula
        if hasattr(event, 'keysym') and event.keysym == "Return":
            self.var.set(self.formula)

In [4]:
class SpreadSheet(tk.Frame):
    '''Creates spreadsheet frame and populates with cell widgets'''
    def __init__(self, rows, cols, wid, ht, frame_row, frame_col, stky, bgcolor, master):
        super().__init__(master)
        self.rows = rows
        self.cols = cols
        self.cells = {}
        self.cellframe = tk.Frame(master, width=wid, height=ht, bg=bgcolor)
        self.cellframe.grid(row=frame_row, column=frame_col, sticky=stky)
        self.create_spreadsheet()

    def create_spreadsheet(self):
        #Column labels
        lab = ('PN','Qty','Date')
        for j in range(self.cols):
            label = tk.Label(self.cellframe, text=lab[j])
            label.config(bg='white')
            label.grid(row=0, column=j+1)
        #Fill in the rows
        for i in range(self.rows):
            rowlabel = tk.Label(self.cellframe, text=str(i + 1))
            rowlabel.config(bg='white')
            rowlabel.grid(row=1+i, column=0)
            for j in range(self.cols):
                cell = Cell(i, j, self.cells, self.cellframe)
                self.cells[cell.name] = cell
                cell.widget.grid(row=1+i, column=1+j)

In [5]:
class ScrolledText(tk.Frame):
    '''Creates scrollable text widget - used for displaying progress & error messages'''
    def __init__(self, wid, ht, row, master):
        super().__init__(master)
        self.width = wid
        self.height = ht
        self.row = row
        self.contents = ''
        self.makewidgets()
        
    def makewidgets(self):
        self.text = tkst.ScrolledText(
        wrap     = tk.WORD,
        width    = self.width,
        height   = self.height
        )
        self.text.grid(row = self.row)
         

In [6]:
def schedule(main_left,status_frame):
    '''runs all functions of scheduling program - will return optimized
       PN list'''
    init_list = main_left.cells.get('A1').copy()
    listy = validate_data(init_list)
    lt = lost_time_matrix(listy,status_frame)
    optimization(lt,listy,status_frame)

In [7]:
def lookup_tables(machine):
    '''Generates and returns tables for use in lost time calculations. 
       This is based on the specific machine being scheduled.'''
    carb_and_hard_push = push_change
    harden_only_push = push_change
    process_change = 0
    quench_setup = 0
    machine_pn_list = scc_list
    return carb_and_hard_push, harden_only_push, process_change, quench_setup, machine_pn_list

In [8]:
def validate_data(init_list):
    '''Takes init_list, removes '-' characters and confirms that pn
       can run on a given machine. Appends valid parts to listy; appends
       invalid parts to listn'''
    pn_list = []
    listy = []
    listn = []
    
    for entry in init_list:
        pn = entry[0]
        entry_val = pn.replace("-","")
        pn_list.append(entry_val) 
    
    for entry in pn_list:
        i=0
        for pn in scc_list: 
            if entry == pn:
                listy.append(entry)
                i = i + 1
            else:
                continue
        if i == 0:
            listn.append(entry)
    return listy


In [9]:
def lost_time_matrix(list,status_frame):
    '''Runs lookup_tables function to create pn-specific lookup tables.
       Returns an (n+1)x(n+1) matrix, lost time for PN changes. The
       +1 is to create a 'dummy node'. This is so that the TSP problem
       can start and end at the dummy node (allows non-dummy PNs to start
       and end at different nodes)'''
    matrix = []
    list.append('x')
    n = len(list)
    for pn1 in list:
        mrow = []
        for pn2 in list:
            process_lt = random.randrange(0, 60, 10) #comparison logic for process change goes here!!!
            push_time_lt = pd.read_csv('push_change.csv', index_col = 'Label').loc['6m','10m']
            quench_lt = process_lt = random.randrange(0, 60, 10) #comparison logic for quench setup change goes here!!!
            if pn1 == pn2:
                pn1_pn2_lt = 9999
            elif pn1 is 'x':
                pn1_pn2_lt = 0
            elif pn2 is 'x':
                pn1_pn2_lt = 0
            else:
                pn1_pn2_lt = process_lt + push_time_lt + quench_lt
            mrow.append(pn1_pn2_lt)
        matrix.append(mrow)
    
    lt = pd.DataFrame(matrix)
    print(lt)
    return lt

In [10]:
def optimization(lt, list, status_frame):
    '''Defines and runs linear integer optimization model for TSP 
       problem. Returns a sorted list of part numbers.This is 1st try 
       @ optimization - appears close to 'proper' setup, but subtour
       elimination constraint causes issues with calculation time'''
    
    model = LpProblem("Minimize Lost Time", LpMinimize)
    n = len(list)
    list=range(n)
    
    #define decision variables
    x = LpVariable.dicts('X', [(pn1, pn2) for pn1 in list for pn2 in list], cat='Binary')
    u = LpVariable.dicts('U', [pn1 for pn1 in list], lowBound=0, upBound=(n-1), cat='Integer')

    #define objective function
    model += lpSum([lt.iloc[pn1, pn2] * x[(pn1, pn2)] for pn1 in list for pn2 in list])
    
    #define constraints
    for pn2 in list:
        model += lpSum([x[(pn1, pn2)] for pn1 in list]) == 1
    for pn1 in list:
        model += lpSum([x[(pn1, pn2)] for pn2 in list]) == 1
    for pn1 in list:
        for pn2 in list:
            model += lpSum([u[pn1] - u[pn2] + n*x[(pn1, pn2)]]) <= n-1
    
    model.solve()
    status_frame.text.insert(INSERT,'Solution: ' + pulp.LpStatus[model.status] + '\n')
    status_frame.text.insert(INSERT, pulp.value(model.objective))
    
    v_list, k, l, b = ([] for i in range(4))
    
    for v in model.variables():
        if v.varValue == 1:
            v_list.append(v.name)
    
    for a in list:
        for i in list:
            for j in list:
                if x[(i, j)].name == v_list[a]:
                    k.append(i)
                    l.append(j)
    
    q = k.index(max(k))
    while (len(b) < n-2):
        for a in range(n):
            if l[a] == k[q]:
                q = l.index(l[a])
                b.append(k[q])
        a = 0 
    
    status_frame.text.insert(INSERT, '\n')
    status_frame.text.insert(INSERT, b)
    
            

In [11]:
def update_recipes(status_frame):
    '''Parses Oracle database and returns most recent recipe data.
       Adds these values to a Pandas dataframe and saves as csv'''
    #file name for saving purposes
    file_name = input('Enter desired file name (.csv not needed):')

    #connection details
    con = cx_Oracle.connect('HTIS_READER/Us3rR3c1peR3$d0nly@z1pi.cis.cat.com')
    cur = con.cursor()

    #getting count data for tqdm loop
    status_frame.text.insert(INSERT,'Getting Counts...\n')
    cur.execute('select count(*) from RECIPE_DB.HTIS_RECIPE where OPERATION in (90,99,120,131) and AREA in (1,3,4,46)')
    t_count = cur.fetchone()
    count = int(t_count[0])

    #getting data from selected operation/area
    status_frame.text.insert(INSERT,'Getting Records...\n')
    cur.execute('select * from RECIPE_DB.HTIS_RECIPE where OPERATION in (90,99,120,131) and AREA in (1,3,4,46)')

    #adding queried data to list of lists; list = [[record1 data], [record2 data], etc]
    status_frame.text.insert(INSERT, 'Adding Records to List...\n')
    list = []
    pbar = tqdm(total = count)
    for result in cur:
        list.append(result)
        pbar.update(1)
    pbar.close()

    #converting row lists to column lists
    status_frame.text.insert(INSERT,'Converting to Columns...\n')
    PART_NO = []
    OPERATION = []
    AREA = []
    CLASS = []
    REVISION = []
    GEN_VAR_NM = []
    GEN_VAR_VAL = []
    for sublist in list:
        PART_NO.append(sublist[0])
        OPERATION.append(sublist[1])
        AREA.append(sublist[2])
        CLASS.append(sublist[3])
        REVISION.append(sublist[4])
        GEN_VAR_NM.append(sublist[5])
        GEN_VAR_VAL.append(sublist[6])

    #zipping list and converting to pd.DataFrame
    zippedlist = zip(PART_NO, OPERATION, AREA, CLASS, REVISION, GEN_VAR_NM, GEN_VAR_VAL)
    df = pd.DataFrame(zippedlist, columns = ['PART_NO', 'OPERATION', 'AREA','CLASS', 'REVISION', 'GEN_VAR_NM', 'GEN_VAR_VAL'])
    cur.close()
    con.close()

    #saving file
    status_frame.text.insert(INSERT,'Saving...\n')
    df.to_csv('recipes.csv')

In [12]:
#define variables, title, and GUI size
scc_list = ['4266842', '5899456', '2671698', '3293299', '2349932', '4949938', '4933384', '5619453', '5804462']
def hello():
    print("hello!")

In [13]:
def create_buttons(self, col, wid, ht, color, x, y, main_left, status_frame):
    '''Creates center-middle frame, with several button widgets. Widgets
       are linked to Scheduling and Recipe Update Commands. Not 100%
       finished...'''
    self.cellframe = tk.Frame(self, width=wid, bg=color, height=ht, padx=x, pady=y)
    self.cellframe.grid(row=0, column=1, sticky="nsew")
    self.cellframe.grid_columnconfigure(1, weight=1)
    #create button to run scheduler and assign run command
    var = IntVar()       
    btn1 = Button(self.cellframe, text="Run Scheduler", command = lambda: schedule(main_left, status_frame))
    btn1.grid(row=0, column=0, padx=10, pady=10, sticky='w')
    #create button to update recipe data and assign run command
    btn2 = Button(self.cellframe, text="Update Recipes", command = lambda self=main_left.cells: self.copy())
    btn2.grid(row=1, column=0, padx=10, pady=10, sticky='w')
    style = Style(self.cellframe) 
    style.configure("TRadiobutton", background = "white", foreground = "black", font = ("arial", 10, "bold"))
    #create tuple to store machine names
    equip = ('SCC', 'Pusher', 'Carb C', 'Reheat')
    #Generate radio buttons for machine list
    for i in range(0,4,1):
        rad = Radiobutton(self.cellframe, text=equip[i], value=i, variable=var)
        rad.grid(column=1, row=i, padx=10, pady=10, sticky='w')

In [ ]:
def build_gui(master=None):
    '''Creates frames/grids, runs spreadsheet and status-box widget
       generation functions, and creates gui'''
    
    #define application colors
    color1 = 'white'
    color2 = 'gray'
    color3 = 'white'
    color4 = 'gray'

    menubar = Menu(root)
    root.config(menu=menubar)

    # create file menu
    filemenu = Menu(menubar, tearoff=0)
    filemenu.add_command(label="Export", command=hello)
    filemenu.add_command(label="Print", command=hello)
    filemenu.add_separator()
    filemenu.add_command(label="Exit", command=root.quit)
    menubar.add_cascade(label="File", menu=filemenu)

    # create edit menu
    editmenu = Menu(menubar, tearoff=0)
    editmenu.add_command(label="Cut", command=hello)
    editmenu.add_command(label="Copy", command=hello)
    editmenu.add_command(label="Paste", command=hello)
    menubar.add_cascade(label="Edit", menu=editmenu)
    
    #create help menu
    helpmenu = Menu(menubar, tearoff=0)
    helpmenu.add_command(label="About", command=hello)
    menubar.add_cascade(label="Help", menu=helpmenu)

    #create main and status frames in root
    main_frame = tk.Frame(root, bg=color1, width=800, height=400, padx=3, pady=3)
    main_frame.grid(row=0, sticky="nsew")
    main_frame.grid_columnconfigure(0, weight=1)
    status_frame = ScrolledText(100,7,4,root)
    
    #split main frame into 3 parts: 2 spreadsheets and 1 button area
    main_left = SpreadSheet(20, 3, 200, 425, 0, 0, 'nw', color1, main_frame)    
    main_mid = create_buttons(main_frame, 1, 400, 425, color1, 3, 3, main_left, status_frame)
    main_right = SpreadSheet(20, 3, 200, 425, 0, 2, 'nw', color1, main_frame) 

build_gui()
root.mainloop()

readonly
      0     1     2     3     4     5     6     7     8     9
0  9999    90   130   110   130   110    50    90    90     0
1    50  9999   110   130    50   110   110    50    30     0
2    70    90  9999    90    90   110   110    90    50     0
3    70    90    30  9999    30    30    30    70    30     0
4    70    50    70   130  9999   110    50    50    90     0
5    30    70    70   110   130  9999   130    70    50     0
6    90    30    50    70    50   130  9999   110    30     0
7    30    90   110    90    90    70    30  9999   130     0
8    50    30    70    50    50    50    70    70  9999     0
9     0     0     0     0     0     0     0     0     0  9999


C:\ProgramData\Anaconda3\lib\site-packages\pulp\pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-13-153b56d99fcd>", line 7, in <lambda>
    btn1 = Button(self.cellframe, text="Run Scheduler", command = lambda: schedule(main_left, status_frame))
  File "<ipython-input-6-0681b619e094>", line 5, in schedule
    optimization(lt,listy,status_frame)
  File "<ipython-input-10-e6920a47ec16>", line 38, in optimization
    if x[(i, j)].name == test[a]:
IndexError: list index out of range
